In [22]:
# import requests
# import ipykernel
# import re
# from notebook.notebookapp import list_running_servers
# from notebook import notebookapp

# from matplotlib import pyplot as plt
# servers = list(notebookapp.list_running_servers())

# TOKEN = servers[0]['token']

# base_url = next(list_running_servers())['url']
# r = requests.get(
#     url=base_url + 'api/sessions',
#     headers={'Authorization': 'token {}'.format(TOKEN),})

# r.raise_for_status()
# response = r.json()

# kernel_id = re.search(
#     'kernel-(.*).json',
#     ipykernel.connect.get_connection_file()
# ).group(1)

# NOTEBOOK_PATH = {
#     r['kernel']['id']: r['notebook']['path']
#     for r in response
# }[kernel_id]
# print(NOTEBOOK_PATH)

KeyError: '21ba9375-98f8-4b82-b27e-7f173007056e'

In [17]:
import os
import glob
import pickle
from itertools import chain
from collections import Counter
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import fitz
from dodfminer.extract.polished.acts.aposentadoria import Retirements
import unidecode

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split, StratifiedKFold

from sklearn.model_selection import  cross_validate

from sklearn.metrics import plot_confusion_matrix
# from sklearn.metrics import f1_score, accuracy_score
from sklearn.metrics import mean_absolute_error, mean_squared_error
import xgboost as xgb
from sklearn.pipeline import Pipeline

In [10]:
%%time
pipe = Pipeline([
    ('vectorizer', TfidfVectorizer()),
    ('model', xgb.XGBClassifier(
        objective='multi:softprob',
        random_state=0,
        num_class=2
    ))
])

CPU times: user 175 µs, sys: 50 µs, total: 225 µs
Wall time: 231 µs


In [11]:
%%time
df = pd.read_csv('blocos_qtd_entidades.csv', index_col=False)
df['y'] = (df.qtd_ents > 0).astype(int)
df.head(2)

CPU times: user 1.17 s, sys: 214 ms, total: 1.38 s
Wall time: 1.54 s


,x0,x,y0,y1,text,qtd_ents,y
0,144.846329,190.436508,659.465149,208.861298,"ANO XLVII EDICAO No- 248\nBRASILIA - DF, TERCA...",0,0
1,56.702831,353.227753,344.850311,370.024231,"Secretaria de Estado de Trabalho, Desenvolvime...",0,0


In [12]:
# train_test_split(df.Text)

In [14]:
%%time

pipe = Pipeline([
    ('vectorizer', TfidfVectorizer()),
    ('model', xgb.XGBRegressor(
        objective='reg:squarederror',
        random_state=42,
        n_jobs=-1,
        
    )),
    
#     ('model', xgb.XGBClassifier(
#         objective='multi:softprob',
#         random_state=42,
#         num_class=2,
#         n_jobs=-1
#     )),
])    

CPU times: user 420 µs, sys: 103 µs, total: 523 µs
Wall time: 541 µs


In [25]:
%%time
skf = StratifiedKFold(n_splits=5)
pipes = []
scores = {
    'mae': [],
    'msae': [],
}
for train_index, test_index in skf.split(df.text, df.y):
    x_train, x_test = df.text[train_index], df.text[test_index]
    y_train, y_test = df.qtd_ents[train_index], df.qtd_ents[test_index]

    pipe = Pipeline([
        ('vectorizer', TfidfVectorizer()),
        ('model', xgb.XGBRegressor(
            objective='reg:squarederror',
            random_state=42,
            n_jobs=-1,

        )),
    ])
    pipe.fit(x_train, y_train)

    y_pred = pipe.predict(x_test)
    scores['mae'].append(mean_absolute_error(y_test, y_pred))
    scores['msae'].append(mean_squared_error(y_test, y_pred))
    pipes.append(pipe)
#     break
# y_pred = pipe.predict(df_test.text)

CPU times: user 25min 1s, sys: 10.1 s, total: 25min 11s
Wall time: 25min 12s


In [26]:
scores

{'mae': [0.009181283275773392,
  0.01218282581664885,
  0.014890245705434721,
  0.011848984107481152,
  0.008639870786284098],
 'msae': [0.025176130697841477,
  0.03220102762547463,
  0.06531995637920182,
  0.037993706655144927,
  0.01781413328532357]}

In [27]:
np.mean(scores['mae']), np.mean(scores['msae'])

(0.011348641938324444, 0.035700990928597284)

In [30]:
%%time
d0_pred=pd.DataFrame({
    'y_pred':pipes[0].predict(df.text),
    'y': df.qtd_ents
})
d0_pred

CPU times: user 12 µs, sys: 0 ns, total: 12 µs
Wall time: 20.7 µs


,y_pred,y
0,-0.000148,0
1,-0.000148,0
2,-0.000148,0
3,-0.000148,0
4,-0.000148,0
...,...,...
104018,-0.000148,0
104019,-0.000148,0
104020,-0.000148,0
104021,-0.000148,0


In [39]:
mask = (np.around(d0_pred.y_pred).astype(int) == d0_pred.y)
d0_err = d0_pred[~mask].copy()
y, y_pred = d0_err.y, d0_err.y_pred

print('(arredondado)mse:', mean_squared_error(y, np.around(y_pred)))
print('(arredondado)mae:', mean_absolute_error(y, np.around(y_pred)))

print('(nao-arredondado)mse:', mean_squared_error(y, y_pred))
print('(nao-arredondado)mae:', mean_absolute_error(y, y_pred))


(arredondado)mse: 5.0606060606060606
(arredondado)mae: 1.7272727272727273
(nao-arredondado)mse: 5.241901407777309
(nao-arredondado)mae: 1.6699442399872675


In [41]:
print("Desvio padrão das quantidades (arredondadas) erroneamente preditas: ", np.std( y - np.around(y_pred) ))

Desvio padrão das quantidades (arredondadas) erroneamente preditas:  2.2148177835904974


In [23]:
prf = 'pipe_xgb_regressor'
pickle.dump(pipe, open(f'{prf}.pkl', 'wb'))
open(f'{prf}.txt', 'w').write(
    f"notebook de origem: xgb-counter.ipynb\n\n"
    f"`{prf}.pkl` contém um pipeline que contém uma instância\n"
    "treinada do modelo xgboost. A entrada é um vetor de strings\n"
    "e a saída um vetor de inteiros , 1-> verdadeiro 0-> falso\n"
    "(no que diz respeito à presença ou não de atos de aposentadoria).\n"
)

292